In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import optuna

from common import EP

import types

Using TensorFlow backend.


In [3]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
# mytrial = []
mytrial = list(pd.read_pickle('../trial/knn.pkl').T.to_dict().values())
df_trial = pd.DataFrame(mytrial)
len(mytrial)

11

In [11]:
KNeighborsRegressor?

In [33]:
param = {'algorithm': {'cls': 'KNeighborsRegressor',
  'fit': {},
  'init': {'n_neighbors': 500,
   'weights': 'uniform',
   'algorithm': 'ball_tree',
   'leaf_size': 30,
   'p': 2,
   'metric': 'minkowski'}},
 'columns': ['q25_roll_std_100',
 'abs_q25_5',
 'iqr_6',
 'abs_q01_4',
 'abs_q75_7',
 'spkt_welch_density__coeff_3',
 'spkt_welch_densitycoeff_2'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'}}

In [34]:
# run one try
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial)

In [35]:
df_trial = pd.DataFrame(mytrial)
df_trial['param-kfold'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial[['datetime','param-kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,param-kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-14 07:06:55.133163,stratified,165,1.594278,0.000007,1.659427,0.000427,0.065149
1,2019-05-14 07:45:51.892280,stratified,165,0.000000,0.000000,1.551405,0.000376,1.551405
2,2019-05-14 08:31:18.107102,stratified,165,1.901755,0.000011,1.922011,0.000492,0.020256
3,2019-05-14 09:09:14.664843,stratified,165,1.866532,0.000010,1.891379,0.000465,0.024846
4,2019-05-14 11:00:33.953643,stratified,165,1.808092,0.000010,1.841374,0.000469,0.033281
5,2019-05-16 12:03:50.123644,stratified,165,1.859466,0.000013,1.893464,0.000111,0.033998
6,2019-05-16 12:51:07.029513,group,165,1.761057,0.001735,2.138490,0.007740,0.377432
7,2019-05-16 13:55:26.659867,timeseries,165,1.757171,0.031389,2.280267,0.231583,0.523096
8,2019-05-16 14:34:08.494798,timeseries,165,1.871639,0.035481,2.258860,0.236903,0.387221
9,2019-05-16 14:40:53.664780,timeseries,165,1.971261,0.037181,2.238941,0.244534,0.267680


In [28]:
df_test_pred = df_trial.loc[4]['df_test_pred']

In [30]:
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission_knn_4.csv', index=False)

In [36]:
df_trial = pd.DataFrame(mytrial)
df_trial.to_pickle('../trial/knn.pkl')